### <center>M-measure - GeNorm

$$ A_{jk} = \left(log_{2}\frac{a_{1j}}{a_{1k}}, \cdots, log_{2}\frac{a_{mj}}{a_{mk}}\right)$$
<br>
$$ V_{jk} = \sigma (A_{jk})$$ 
<br>
$$ M_{j} = \frac{\sum_{k=1}^{n}V_{jk}}{n-1} $$

In [71]:
import numpy as np
import pandas as pd
from time import time

In [5]:
data = pd.read_csv("./data/all_counts_9_norm_rpkm_log2_preprocessed.csv", index_col=0)
data.head()

,BB9,BB10,BB17,BB19,BB20,BB21,BB11,BB12,BB18
gene,,,,,,,,,
aaeA,5.542667,5.876225,5.189101,6.178741,6.258982,5.921482,6.565341,6.308676,6.256590
aaeB,5.627977,5.436188,5.397088,5.493631,5.603751,5.716862,5.840627,6.132819,5.896502
aaeR,5.645506,5.758852,5.107190,5.824890,5.487705,6.033795,5.710782,6.302440,16.270368
aaeX,6.081770,6.043589,6.030808,6.358792,15.682419,6.129392,6.042647,6.134309,15.797839
aas,6.224669,6.297153,6.297008,6.047021,6.038410,6.296795,6.295251,6.598525,6.568365


In [87]:
def get_M_measure(d):
    """
    compute the M-measure given by GeNorm
    d: DataFrame
    """
    data = d.copy()
    m_measure = {}
    for idx in range(data.shape[0]):
        all_pairs = np.std(np.log2((data.iloc[idx,:]/data).loc[data.index!=data.index[idx]]), axis=1)
        m_measure[data.index[idx]] = np.mean(all_pairs)
    
    return dict(sorted(m_measure.items(), key=lambda x: x[1]))

In [88]:
t_i = time()
m = get_M_measure(data)
t_f = time()
print("avg time:", t_f-t_i, "[s]")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log2
  if __name__ == '__main__':


avg time: 22.928868293762207 [s]


### M-measure for the reference genes candidates of the ICCSA conference

In [94]:
candidates = pd.read_csv("./data/ICCSA_cantidates_06_03_20_v1.csv", index_col=0)
candidates.head()

,BB9,BB10,BB17,BB19,BB20,BB21,BB11,BB12,BB18
gene,,,,,,,,,
aaeA,-1.021960,-0.979435,-1.070980,-0.936075,-0.964250,-1.031994,-0.934337,-0.939543,-0.937269
aas,-0.922192,-0.918634,-0.912901,-0.955073,-0.996782,-0.974946,-0.973578,-0.897515,-0.892211
accB,-0.514109,0.918962,1.020704,0.785425,0.781750,0.679621,0.702075,0.594880,0.713118
aceB,0.857671,0.777676,0.822172,0.077504,0.069610,-0.091237,-0.130144,1.031624,-0.112132
aceE,0.966515,0.934333,1.030230,0.849922,0.855701,0.749005,0.780537,0.621839,0.832078


In [96]:
obj_rg = open("./data/reference_genes.txt")
rg = obj_rg.read().splitlines()
rg.remove("idnT")
print("quantidade de genes de referência:", len(rg))
print(rg)

quantidade de genes de referência: 20
['cysG', 'hcaT', 'rrsA', 'ihfB', 'ssrA', 'gyrA', 'recA', 'rpoB', 'rpoA', 'gyrB', 'rho', 'ftsZ', 'secA', 'rpoC', 'gmk', 'adk', 'rpoD', 'dnaG', 'glnA', 'recF']


In [97]:
class scaler:
    def __init__(self, xmin, xmax):
        """
        minmax scaler from dataframe
        """
        self.xmin = xmin
        self.xmax = xmax
        self.min_data = False
        self.max_data = False
        self.flag = False
        
    def fit(self, X):
        self.min_data = np.min(X).values
        self.max_data = np.max(X).values
        self.flag = True
        
    def transform(self, X):
        assert self.flag, "Erro de treinamento, primeiro tem que treinar o Scaler"
        X_r = X.copy()
        X_r = ((X_r - self.min_data)/(self.max_data - self.min_data))*(self.xmax-self.xmin) + self.xmin
        return X_r
    
    def inverse_transform(self, X):
        assert self.flag, "Erro de treinamento, primeiro tem que treinar o Scaler"
        X_r = X.copy()
        X_r = ((X_r - self.xmin)*(self.max_data - self.min_data)/(self.xmax - self.xmin)) + self.min_data
        return X_r

In [100]:
X_rg = data.loc[rg]
obj = scaler(-1,1)
obj.fit(X_rg)

In [102]:
get_M_measure(obj.inverse_transform(candidates))

{'orn': 0.18020512087348972,
 'mtlR': 0.18030703717700128,
 'yqiB': 0.1803496660187161,
 'nrdD': 0.18038164552222996,
 'kbl': 0.1804281614628066,
 'waaQ': 0.18051045203242003,
 'lptG': 0.1805138416823366,
 'gss': 0.1805853104857115,
 'ftsX': 0.18060773077057332,
 'dsbC': 0.18075076925663106,
 'xerD': 0.18075289997236635,
 'insB1': 0.18076369274927326,
 'chrR': 0.18077660884523436,
 'ygeR': 0.1808562820779858,
 'gntX': 0.18102468461048807,
 'hflX': 0.18113836130869992,
 'selB': 0.18115009612151706,
 'rplI': 0.1811758565450173,
 'yicC': 0.18121156374437852,
 'epmB': 0.1812418966280034,
 'yoaH': 0.18124543333057183,
 'insC1': 0.18125196782463188,
 'holC': 0.1812723877107782,
 'hdfR': 0.1812774627916586,
 'yjgA': 0.1812996234161826,
 'yhfK': 0.18130257196780014,
 'envZ': 0.18131637761889693,
 'xylR': 0.1814203423530188,
 'ygiC': 0.1814318315871728,
 'insL1': 0.18147287300492515,
 'katG': 0.18151123619437334,
 'waaF': 0.18155149207477175,
 'yjiA': 0.18158089924475337,
 'glyS': 0.18161863566